## Create a pandas DataFrame from a 6 gigabyte text file with over 85 million Top-Of-Book historical bid/ask/trade records for CLK20 crude oil, on 2020-03-12. 

In a single trading day, and a single futures contract, traders will submit about 85 million bids and offers to GLOBEX. This translates to about 1 order every millisecond `(23*60*60*1000/85000000)`.  The CME allows one to purchase this Top Of Book Data, and also provides example files and file formats.   

The page that explains these files is: 
 * https://www.cmegroup.com/confluence/display/EPICSANDBOX/Top+of+Book+-+BBO . 
   * This page explains the overview and file formats of Top Of Book historical data
 
The URL for the crude oil example data on this day is: 
 * https://www.cmegroup.com/trading/market-tech-and-data-services/files/large-downloads/xnym-bbo-cl-fut-20200312-r-00065.csv.gz .  

The gz file above - when expanded - is 6 gigabytes. Each text line contains column values **with no separators**. An html table at the path`./temp_folder/top_book_layout.html` 
contains the beginning and ending indices for each column of data. The pandas DataFrame `df_top_layout` gets created from that html_table using `pd.read_html('./temp_folder/top_book_layout.html')`

Follow the cells below to see how this 6 gigbyte text file can be split into 86 pandas dataframes.

In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import requests
import shutil
import pathlib
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

import glob
from lxml import html
from IPython.display import display
from tqdm.notebook import tqdm


In [2]:
def _firefox_driver(save_folder,show_download_dialog=True):
    options = Options()
    options.headless = not show_download_dialog
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.dir", save_folder)
    profile.set_preference("browser.download.manager.showWhenStarting", show_download_dialog)
    
    if show_download_dialog:
        profile.set_preference("browser.helperApps.alwaysAsk.saveToDisk", "application/x-gzip")
    else:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

    driver = webdriver.Firefox(profile,options=options)    
    return driver

### First, download .gz file with 85+ million rows of BBO data 

In [3]:
# def download_gz_file(url,local_filename):
#     # NOTE the stream=True parameter below
#     with requests.get(url, stream=True) as r:
#         r.raise_for_status()
#         with open(local_filename, 'wb') as f:
#             for chunk in tqdmnb(r.iter_content(chunk_size=8192)): 
#                 # If you have chunk encoded response uncomment if
#                 # and set chunk_size parameter to None.
#                 #if chunk: 
#                 f.write(chunk)
#     return local_filename

# def download_gz_file_slow(url,local_filename):
#     with requests.get(url, stream=True) as r:
#         with open(local_filename, 'wb') as f:
#             shutil.copyfileobj(r.raw, f)


In [4]:
def download_cme_bbo_gz_file(show_download_dialog=True): 
    bbo_url_part1  = "https://www.cmegroup.com/trading/market-tech-and-data-services"
    bbo_url_part2 = "/files/large-downloads/xnym-bbo-cl-fut-20200312-r-00065.csv.gz"
    bbo_url = bbo_url_part1 + bbo_url_part2
    # name of file contained in the gz zipped file after it is expanded
    #  THIS IS NOT A CSV FILE !!!, even if it's suffix is ".csv"
    local_bbo_filename = 'xnym-bbo-cl-fut-20200312-r-00065.gz'
    # the folder into which you expanded the cme gz file that you downloaded
    local_folder = os.path.abspath('./temp_folder/cme_datamine')
    # full path of output file
    local_save_path = f'{local_folder}/{local_bbo_filename}'

#     download_gz_file(bbo_url,cl_example_csv_path)
#     !curl -o {local_save_path} {bbo_url}
    webdr = _firefox_driver(local_save_path,show_download_dialog=show_download_dialog)
    n = datetime.datetime.now().strftime('%Y-%m-5d %H:%M:%S')
    print(f"{n} downloading file from {bbo_url}.")
    print("Hit the stop button after the file has been unzipped")
    webdr.get(bbo_url)
    n = datetime.datetime.now().strftime('%Y-%m-5d %H:%M:%S')
    print(f"{n} finished downloading file")
    
    webdr.quit()

In [124]:
download_cme_bbo_gz_file()


2021-10-5d 11:50:30 downloading file from https://www.cmegroup.com/trading/market-tech-and-data-services/files/large-downloads/xnym-bbo-cl-fut-20200312-r-00065.csv.gz.
Hit the stop button after the file has been unzipped


KeyboardInterrupt: 

In [5]:
df_top_layout = list_df_layouts = pd.read_html(
    './temp_folder/top_book_layout.html'
)[0]
df_top_layout

,Field Number,Data Field,Start Position,End Position,Length,Description
0,1,Trade Date,1,8,8,YYYYMMDD- Day the trade or quote was entered
1,2,Trade Time,9,14,6,HHMMSS- Time the trade or quote was entered in...
2,3,Trade Sequence Number,15,22,8,######## - sequence the quote or trade was ent...
3,4,Session Indicator,23,23,1,(R/E) Indicates the Regular (PIT) or Electroni...
4,5,Ticker Symbol,24,26,3,The product code
5,6,FOI Indicator,27,27,1,Futures (F) / Options (O) - Indicates the type...
6,7,Delivery Date,28,31,4,(YYMM) Indicates the month the contract expires
7,8,Trade Quantity,32,36,5,Number of contracts available for trade or traded
8,9,Strike Price,37,43,7,"The strike or exercise price of the option, if..."
9,10,Strike Price Decimal Locator,44,44,1,Decimal place indicator for strike price


In [8]:
if __name__=='__main__':    
    # name of file contained in the gz zipped file after it is expanded
    #  THIS IS NOT A CSV FILE !!!, even if it's suffix is ".csv"
    local_bbo_filename = 'xnym-bbo-cl-fut-20200312-r-00065.csv'
    # the folder into which you expanded the cme gz file that you downloaded
    local_folder = os.path.abspath(
        f"./temp_folder/cme_datamine/{local_bbo_filename.replace('csv','gz')}"
    )
    # full path of output file
    local_save_path = f'{local_folder}/{local_bbo_filename}'
    cl_example_csv_path = local_save_path

    # read all of the lines
    print(f'{datetime.datetime.now()} - Read all lines from {local_save_path}.  This might take some time')
    cme_top_lines = open(cl_example_csv_path,'r').readlines()
    print(f"{datetime.datetime.now()} - Finished reading all lines.")

    # path to write each "partition" DataFrame
    #  for cme_top_lines[beg_index:end_index], on each iteration of the loop below
    output_path = './temp_folder/cme_datamine/df_clk20_tob_PART.csv'

    # column indices for df_top_layout
    data_field_col = 1 # column number (0 offset) for the field name of the data
    start_col = 2 # column number for the starting character of the data
    end_col = 3 # column number for the ending character of the data

    # partition_size determines the number of lines that you will read from cme_top_lines
    partition_size = 1000000
    # number of partitions in cme_top_lines
    partitions = int(len(cme_top_lines)/partition_size) + 1

    for partition in tqdm(range(partitions)):
        # get beginning index of this partition
        beg_index = partition * partition_size
        # get ending index of this partition
        end_index = beg_index + partition_size

        # create a dictionary of every all the data for each field that is defined
        #   in each row of df_top_layout
        dict_top = {
            r[data_field_col]:[
                l[r[start_col]-1:r[end_col]]
                for l in cme_top_lines[beg_index:end_index]
            ]
            for r in df_top_layout.values 
        }

        # create a save path for this partition
        this_path = output_path.replace('PART',f'000{partition}'[-2:])

        # create a DataFrame from dict_top, and write it
        pd.DataFrame(dict_top).to_csv(this_path,index=False)
    

2021-10-27 12:15:05.470765 - Read all lines from /Users/bperlman1/Documents/billybyte/pyliverisk/cme_datamine_api/temp_folder/cme_datamine/xnym-bbo-cl-fut-20200312-r-00065.gz/xnym-bbo-cl-fut-20200312-r-00065.csv.  This might take some time
2021-10-27 12:15:24.197804 - Finished reading all lines.


  0%|          | 0/86 [00:00<?, ?it/s]